Synonyms in the new ModelResult
---

This notebook is meant to demo how synonyms should work in the new `ModelResult` prototype. First, we import the prototype functionality and initialize a model which only has `nbp` in the files.

In [1]:
from ModelResult import ModelResult
clm4 = ModelResult("/home/nate/data/ILAMB/MODELS/CLM/CLM4.0",name="CLM4").findFiles()

As a reference, now we get `nbp` from the model and print it to see the minimum/maximum/mean values.

In [2]:
nbp = clm4.getVariable("nbp")
print(nbp)

Variable: nbp
-------------
                unit: kg m-2 s-1
          isTemporal: True (1932)
           isSpatial: True (192,288)
           isLayered: False
          nDatasites: N/A
           dataShape: (1932, 192, 288)
             dataMax: 5.861439e-08
             dataMin: -1.318722e-07
            dataMean: -5.798144e-11



However, variable names in ILAMB are case sensitive. So if we try to get `NBP` this will throw an exception that we catch and report to the user.

In [3]:
try:
    nbp = clm4.getVariable("NBP")
    print("Success")
except Exception as e:
    print(e)

Cannot find 'NBP' in 'CLM4'


One way to do this is to give synonyms when asking the model for the variable. This is how ILAMB works now. 

In [4]:
nbp = clm4.getVariable("NBP",synonyms=["nbp"])
print(nbp)

Variable: nbp
-------------
                unit: kg m-2 s-1
          isTemporal: True (1932)
           isSpatial: True (192,288)
           isLayered: False
          nDatasites: N/A
           dataShape: (1932, 192, 288)
             dataMax: 5.861439e-08
             dataMin: -1.318722e-07
            dataMean: -5.798144e-11



The problem with this method is that these synonyms are not global. So for example, if my model calls downward radiation `fsds`, when I need to get `rsds` I can simply provide the synonym. However, if I go to compute albedo by derived expression `albedo=rsus/rsds`, I cannot specify alternates for each free symbols in the expression. What we need rather is a method to define a global synonym by model. This is now possible by the following:

In [5]:
clm4.addSynonym("NBP=nbp")
nbp = clm4.getVariable("NBP")
print(nbp)    

Variable: NBP
-------------
                unit: kg m-2 s-1
          isTemporal: True (1932)
           isSpatial: True (192,288)
           isLayered: False
          nDatasites: N/A
           dataShape: (1932, 192, 288)
             dataMax: 5.861439e-08
             dataMin: -1.318722e-07
            dataMean: -5.798144e-11



In the above example, anywhere `NBP` is used with respect to the `clm4` model, the expression will be used instead. This will take care of some issues we have had with ELM. We can simply add a model synonym `nbp=-nee`. Even if `nbp` is a variable, ILAMB will prioritize the model-defined synonyms over the variables in the files.

For example, in the code below I am trying to extract `NBP` which is not in the model files. But there is a synonym defined in the `getVariable()` call allowing `nbp` to be used which *is* in the model files. However we have also a model synonym defined. The model synonym will be used as seen by the signs of the minimum/maximum/mean of the variable.

In [6]:
# re-initialize to clear the synonyms
clm4 = ModelResult("/home/nate/data/ILAMB/MODELS/CLM/CLM4.0",name="CLM4").findFiles()

# flipping sign just to show which gets used
clm4.addSynonym("NBP=-nbp") 
nbp = clm4.getVariable("NBP",synonyms=["nbp"])

# based of the signs of the min/max/mean you can see the model synonym was preferred
print(nbp)

Variable: NBP
-------------
                unit: (kg m-2 s-1)
          isTemporal: True (1932)
           isSpatial: True (192,288)
           isLayered: False
          nDatasites: N/A
           dataShape: (1932, 192, 288)
             dataMax: 1.318722e-07
             dataMin: -5.861439e-08
            dataMean: 5.798144e-11

